# Lab 1 report

In [99]:
from nltk.corpus import shakespeare
from xml.etree import ElementTree
import nltk
import re
import random
import numpy as np
from nltk.corpus import stopwords

nltk.download('shakespeare')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package shakespeare to /root/nltk_data...
[nltk_data]   Package shakespeare is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Task 1

I went through the data once, extracting all the information I'll need in the following tasks, storing it mostly in `play_info` dictionary, which contained `Play_info` class accesible through the play's name.

Data for Task 3 is stored in `dream_lines_by_character` dictionary.

I used the `xml` files instead of the provided text files.

In [100]:
class Play_info:
    def __init__(self, name : str,
                 tokens : list[str],
                 types : set,
                 spoken_lines : list[str],
                 lines_per_act : list[int]
                 ) -> None:
        self.name = name
        self.tokens = tokens
        self.types = types
        self.spoken_lines = spoken_lines
        self.lines_per_act = lines_per_act

In [101]:
play_names = shakespeare.fileids()
play_info : dict[str, Play_info] = {}

dream_lines_by_character = {}
dream_speaker = ""

# I am sorry for this disgusting piece of nested code, but considering the nested structure of xml files,
# this is necesary evil.
for play_name in play_names:
    tree = ElementTree.parse(shakespeare.open(play_name))
    root = tree.getroot()

    tokens = []
    spoken_lines = []

    lines_per_act = []

    for book_part in root:

        # ignore parts of the book that aren't part of the play
        if book_part.tag != "ACT":
            continue

        for act_part in book_part:

            # tokenize scene titles and introductory texts outside of scenes
            if act_part.tag != "SCENE" and act_part.text != None:
                tokens.extend(nltk.tokenize.word_tokenize(act_part.text))
                continue

            for scene_part in act_part:

                # tokenize stage directions and descriptions outside of dialogues
                if scene_part.tag != "SPEECH" and scene_part.text != None:
                    tokens.extend(nltk.tokenize.word_tokenize(scene_part.text))
                    continue

                for speech_part in scene_part:

                    # ignore empty speech parts
                    if speech_part.text == None:
                        continue

                    # tokenize dialogue
                    tokens.extend(nltk.tokenize.word_tokenize(speech_part.text))

                    # collect spoken lines
                    if speech_part.tag == "LINE":
                        spoken_lines.append(speech_part.text)

                    # separate lines of dream.xml by speaker (3A)
                    if play_name == "dream.xml":
                        if speech_part.tag == "SPEAKER":
                            dream_speaker = speech_part.text
                        if speech_part.tag == "LINE":
                            if dream_speaker not in dream_lines_by_character:
                                dream_lines_by_character[dream_speaker] = []
                            dream_lines_by_character[dream_speaker].append(speech_part.text)

        # cumulative sum of lines by act
        lines_per_act.append(len(spoken_lines))

    # extract types from tokens
    types = set(tokens)
    play_info[play_name] = Play_info(play_name, tokens, types, spoken_lines, lines_per_act)


## Task 2

I looped through the collected data and found the play with the most tokens, types and lines.

In [102]:
max_tokens = 0
max_tokens_play = ""
max_types = 0
max_types_play = ""
max_lines = 0
max_lines_play = ""

for play_name in play_names:
    info : Play_info = play_info[play_name]

    if len(info.tokens) > max_tokens:
        max_tokens_play = info.name
        max_tokens = len(info.tokens)

    if len(info.types) > max_types:
        max_types_play = info.name
        max_types = len(info.types)

    if len(info.spoken_lines) > max_lines:
        max_lines_play = info.name
        max_lines = len(info.spoken_lines)

print(max_tokens)
print(max_tokens_play)
print(max_types)
print(max_types_play)
print(max_lines)
print(max_lines_play)


39133
hamlet.xml
5171
hamlet.xml
3978
hamlet.xml


Hamlet has the most tokens, types and spoken lines. It's by far the longest play, with
- 39133 tokens,
- 5171 types,
- 2978 lines.


## Task 3

The analysis of *A Midsummer Night’s Dream*.

### 3 A

I already extracted the corpus for the lines spoken by each character in Task 1.

The method of extraction can be used for any play in the corpus without knowing the characters’ names in advance.

### 3 B, C, D, E

In the folowing four tasks, we are supposed to answer three questions for four different instances of the corpus of the lines spoken by each character of *A Midsummer Night’s Dream*.

They are:
- Which characters have the most lines?
- What are the twenty most frequent words in the play, and how many times do they appear?
- What are each character’s five most frequently used words?

The functions below answers these questions.

In [103]:
def twenty_most_frequent_words(words_by_character):
    word_counts = {}
    for character, words in words_by_character.items():
        for word in words:
            if word not in word_counts:
                word_counts[word] = 0
            word_counts[word] += 1

    return sorted(word_counts.items(), key=lambda x: - x[1])[:20]

def five_most_frequent_words_by_character(words_by_character):
    word_counts = {}
    for character, words in words_by_character.items():
        word_counts[character] = {}
        for word in words:
            if word not in word_counts[character]:
                word_counts[character][word] = 0
            word_counts[character][word] += 1

    for character in word_counts.keys():
        word_counts[character] = sorted(word_counts[character].items(), key=lambda x: - x[1])[:5]

    return word_counts

I will answer the first question
- Which characters have the most lines?

just once, the number of lines for character stays the same no matter how I preprocess the text.

In [104]:
max_lines = 0
max_lines_character = ""
midsummers_dream_words_by_character = {}

for character, lines in dream_lines_by_character.items():
    if len(lines) > max_lines:
        max_lines = len(lines)
        max_lines_character = character

    midsummers_dream_words_by_character[character] = ' '.join(lines)

print("The character with the most lines:")
print(max_lines)
print(max_lines_character)
print()

The character with the most lines:
232
THESEUS



#### Apply only tokenization

I tried two ways of tokenizing, one way with the official nltk tokenizer, and the other with a regex `\w+`.

When using the official tokenizer, the most usual words are punctuations, which is interesting, but it doesn't give us any information, about how lemmatizing or stemming changes the most frequent words.

That is why I chose to use the regex tokenizer for the most of the task.

**nltk word tokenizer**

In [105]:
midsummers_dream_words_by_character_tokenized = midsummers_dream_words_by_character.copy()

max_lines = 0
max_lines_character = ""

for character, words in midsummers_dream_words_by_character_tokenized.items():
    midsummers_dream_words_by_character_tokenized[character] = nltk.tokenize.word_tokenize(words)

print("TWENTY MOST COMMON WORDS IN THE PLAY:")
print(twenty_most_frequent_words(midsummers_dream_words_by_character_tokenized))
print()

for character, words in five_most_frequent_words_by_character(midsummers_dream_words_by_character_tokenized).items():
    print(character)
    print(words)

TWENTY MOST COMMON WORDS IN THE PLAY:
[(',', 1740), ('.', 667), ('the', 473), ('I', 458), (';', 349), ('and', 335), (':', 275), ('to', 270), ('of', 245), ('you', 239), ('a', 232), ('in', 211), ('And', 196), ('is', 189), ('?', 177), ('me', 176), ('my', 175), ('!', 173), ('not', 173), ('with', 140)]

THESEUS
[(',', 160), ('.', 82), ('the', 67), ('and', 49), (';', 41)]
HIPPOLYTA
[(',', 18), ('.', 13), ('the', 11), ('a', 9), ('I', 8)]
EGEUS
[(',', 45), ('my', 16), ('of', 11), ('.', 11), ('her', 8)]
HERMIA
[(',', 119), ('.', 49), ('I', 44), ('!', 33), ('?', 33)]
DEMETRIUS
[(',', 121), ('.', 53), ('I', 46), ('the', 31), ('not', 25)]
LYSANDER
[(',', 162), ('I', 54), ('.', 52), (';', 38), ('to', 29)]
HELENA
[(',', 203), ('.', 60), ('you', 57), ('I', 54), ('me', 46)]
QUINCE
[(',', 76), ('.', 45), ('you', 29), ('and', 27), ('the', 24)]
BOTTOM
[(',', 170), ('.', 75), ('I', 75), ('the', 54), ('to', 50)]
FLUTE
[(',', 19), ('a', 9), ('.', 7), ('not', 5), ('he', 5)]
STARVELING
[('.', 5), (',', 3), ('

**regex `\w+` tokenizer**

In [124]:
midsummers_dream_words_by_character_tokenized = midsummers_dream_words_by_character.copy()

max_lines = 0
max_lines_character = ""

for character, words in midsummers_dream_words_by_character_tokenized.items():
    midsummers_dream_words_by_character_tokenized[character] = re.findall( r'\w+',words)

print("TWENTY MOST COMMON WORDS IN THE PLAY:")
print(twenty_most_frequent_words(midsummers_dream_words_by_character_tokenized))
print()

for character, words in five_most_frequent_words_by_character(midsummers_dream_words_by_character_tokenized).items():
    print(character)
    print(words)

TWENTY MOST COMMON WORDS IN THE PLAY:
[('the', 473), ('I', 461), ('and', 335), ('to', 275), ('of', 245), ('you', 239), ('a', 233), ('in', 213), ('And', 196), ('is', 186), ('me', 176), ('my', 175), ('not', 160), ('with', 140), ('that', 139), ('s', 125), ('it', 117), ('your', 115), ('this', 115), ('will', 109)]

THESEUS
[('the', 67), ('and', 49), ('of', 40), ('to', 38), ('in', 35)]
HIPPOLYTA
[('the', 11), ('a', 9), ('I', 8), ('in', 6), ('of', 6)]
EGEUS
[('my', 16), ('of', 11), ('her', 8), ('to', 7), ('is', 7)]
HERMIA
[('I', 44), ('me', 29), ('my', 25), ('you', 25), ('to', 22)]
DEMETRIUS
[('I', 46), ('the', 31), ('not', 21), ('to', 18), ('me', 18)]
LYSANDER
[('I', 54), ('to', 30), ('the', 27), ('my', 24), ('of', 22)]
HELENA
[('you', 57), ('I', 54), ('me', 46), ('and', 36), ('the', 29)]
QUINCE
[('you', 29), ('and', 27), ('the', 24), ('a', 19), ('is', 17)]
BOTTOM
[('I', 75), ('the', 54), ('to', 50), ('a', 46), ('and', 40)]
FLUTE
[('a', 9), ('not', 5), ('he', 5), ('I', 4), ('Pyramus', 4)]
ST

#### Apply lemmatization over tokenization

For lemmatizing the text, I used nltk Word Net Lemmatizer.

In [107]:
midsummers_dream_words_by_character_lemmatized = midsummers_dream_words_by_character_tokenized.copy()

for character, words in midsummers_dream_words_by_character_lemmatized.items():
    lemmatized = []
    for word in words:
        lemmatized.append(nltk.WordNetLemmatizer.lemmatize(self=nltk.WordNetLemmatizer ,word=word))

    midsummers_dream_words_by_character_lemmatized[character] = lemmatized

print("TWENTY MOST COMMON WORDS IN THE PLAY:")
print(twenty_most_frequent_words(midsummers_dream_words_by_character_lemmatized))
print()

for character, words in five_most_frequent_words_by_character(midsummers_dream_words_by_character_lemmatized).items():
    print(character)
    print(words)

TWENTY MOST COMMON WORDS IN THE PLAY:
[('the', 473), ('I', 461), ('and', 335), ('a', 328), ('to', 275), ('of', 245), ('you', 239), ('in', 213), ('And', 196), ('is', 186), ('me', 176), ('my', 175), ('not', 160), ('with', 140), ('that', 139), ('s', 125), ('it', 117), ('your', 115), ('this', 115), ('love', 114)]

THESEUS
[('the', 67), ('and', 49), ('of', 40), ('to', 38), ('a', 36)]
HIPPOLYTA
[('the', 11), ('a', 9), ('I', 8), ('in', 6), ('of', 6)]
EGEUS
[('my', 16), ('of', 11), ('her', 8), ('to', 7), ('is', 7)]
HERMIA
[('I', 44), ('me', 29), ('my', 25), ('you', 25), ('to', 22)]
DEMETRIUS
[('I', 46), ('the', 31), ('not', 21), ('to', 18), ('me', 18)]
LYSANDER
[('I', 54), ('a', 31), ('to', 30), ('the', 27), ('my', 24)]
HELENA
[('you', 57), ('I', 54), ('me', 46), ('a', 39), ('and', 36)]
QUINCE
[('you', 29), ('and', 27), ('a', 25), ('the', 24), ('is', 17)]
BOTTOM
[('I', 75), ('the', 54), ('a', 51), ('to', 50), ('and', 40)]
FLUTE
[('a', 11), ('not', 5), ('he', 5), ('I', 4), ('Pyramus', 4)]
STARV

#### Apply stemming over tokenization

I am using nltk Snowball stemmer for English.

In [108]:
midsummers_dream_words_by_character_stemmed = midsummers_dream_words_by_character_tokenized.copy()

stemmer = nltk.SnowballStemmer("english")

for character, words in midsummers_dream_words_by_character_stemmed.items():
    stemmed = []
    for word in words:
        stemmed.append(stemmer.stem(word))

    midsummers_dream_words_by_character_stemmed[character] = stemmed

print("TWENTY MOST COMMON WORDS IN THE PLAY:")
print(twenty_most_frequent_words(midsummers_dream_words_by_character_stemmed))
print()

for character, words in five_most_frequent_words_by_character(midsummers_dream_words_by_character_stemmed).items():
    print(character)
    print(words)

TWENTY MOST COMMON WORDS IN THE PLAY:
[('the', 546), ('and', 531), ('i', 464), ('to', 336), ('you', 274), ('of', 262), ('a', 260), ('in', 240), ('my', 199), ('is', 194), ('that', 186), ('me', 176), ('not', 170), ('with', 168), ('this', 148), ('for', 142), ('love', 136), ('it', 133), ('your', 127), ('s', 125)]

THESEUS
[('the', 82), ('and', 61), ('to', 49), ('of', 45), ('in', 40)]
HIPPOLYTA
[('the', 12), ('a', 9), ('and', 8), ('i', 8), ('of', 7)]
EGEUS
[('my', 18), ('of', 13), ('and', 11), ('to', 8), ('her', 8)]
HERMIA
[('i', 44), ('and', 30), ('me', 29), ('my', 28), ('you', 28)]
DEMETRIUS
[('i', 46), ('the', 35), ('and', 24), ('to', 23), ('not', 22)]
LYSANDER
[('i', 54), ('and', 39), ('to', 34), ('the', 34), ('my', 27)]
HELENA
[('you', 67), ('i', 55), ('and', 54), ('me', 46), ('to', 41)]
QUINCE
[('you', 33), ('and', 27), ('the', 25), ('a', 20), ('is', 18)]
BOTTOM
[('i', 75), ('the', 62), ('to', 51), ('a', 48), ('and', 46)]
FLUTE
[('a', 9), ('not', 5), ('he', 5), ('i', 4), ('most', 4)]


#### Remove stopwords

I am removing stopwords from the tokenized version of the text and using the standard nltk stopwords list.

In [109]:
midsummers_dream_words_by_character_filtered = midsummers_dream_words_by_character_tokenized.copy()

for character, words in midsummers_dream_words_by_character_filtered.items():
    filtered = []
    for word in words:
        if word.lower() not in stopwords.words('english'):
            filtered.append(word)

    midsummers_dream_words_by_character_filtered[character] = filtered

print("TWENTY MOST COMMON WORDS IN THE PLAY:")
print(twenty_most_frequent_words(midsummers_dream_words_by_character_filtered))
print()

for character, words in five_most_frequent_words_by_character(midsummers_dream_words_by_character_filtered).items():
    print(character)
    print(words)

TWENTY MOST COMMON WORDS IN THE PLAY:
[('love', 105), ('thou', 98), ('shall', 65), ('thee', 64), ('thy', 56), ('man', 50), ('night', 49), ('Pyramus', 44), ('Hermia', 43), ('sweet', 43), ('must', 42), ('Demetrius', 40), ('eyes', 39), ('come', 39), ('Lysander', 39), ('see', 38), ('one', 37), ('good', 36), ('would', 36), ('play', 35)]

THESEUS
[('shall', 10), ('love', 7), ('play', 7), ('moon', 6), ('come', 6)]
HIPPOLYTA
[('night', 3), ('moon', 3), ('Four', 2), ('quickly', 2), ('like', 2)]
EGEUS
[('Demetrius', 5), ('lord', 4), ('Lysander', 4), ('love', 4), ('duke', 3)]
HERMIA
[('Lysander', 21), ('thou', 13), ('love', 11), ('eyes', 6), ('low', 6)]
DEMETRIUS
[('thou', 12), ('thy', 10), ('love', 9), ('thee', 9), ('lord', 8)]
LYSANDER
[('love', 21), ('thee', 16), ('Hermia', 14), ('Helena', 13), ('thou', 13)]
HELENA
[('Hermia', 15), ('love', 15), ('Demetrius', 14), ('one', 11), ('eyes', 9)]
QUINCE
[('Pyramus', 15), ('play', 10), ('must', 10), ('Thisby', 8), ('Bottom', 7)]
BOTTOM
[('good', 15), 

### What do you notice about how the answers change? (3F)

There is a really big difference based on the tokenizer which is used, but that really just comes down to the question of what is a word.

From the other experiments, the biggest impact had the stopwords removal for sure. It makes sence, that removing the most used words in a language would greatly impact the top most repeated words of a play in that language.

Second biggest impact had stemming, but that probably is because the stemmer I used also lowercased all of the words. Because of that, *and* and *And* became one word and got a bit higher in the list, along with some other words.

The difference between just tokenizing and tokenizing and lemmatizing is almost nothing.

These observations could be connected to the fact that most of the top words are actually stopwords, and that a lot of stopwords have actually just one form, so their original, stemmed and lemmatized forms are one and the same.

### 3 G

Even though stopwords removal gave the best results, it could've been even better. I was actually trying to find Shakespearian stopwords list, but I wasn't succesful.

As for the second part, many verbs have the form ending with *-st*, e.g. *hast* or *dost*. Those aren't recognized by modern stemmers nor lemmatizers. Another example is with the *-th* ending: *hath*, *doth*. Also the *thou*, *thee* and *thy* aren't known by modern English lemmatizers.

We could add rules for dealing with the verbs, because the same as we now have rules for word endings, there are rules for verb endings from shakespearian times.

## Task 4

N gram model.

In [110]:
class Ngram_model:
    """N gram model.

    Needs to be trained by calling 'fit()' before usage.
    """

    def __init__(self, N : int, unit : str = "word") -> None:
        """N : number of consecutive units taken into account.
         In the range of 1 to number of units in a training test.

        unit = {'word', 'char'} : basic unit for segmenting text"""

        self.N = N
        self.unit = unit
        self.probabilities_dict = {}

        if self.unit != "word" and self.unit != "char":
            raise ValueError("Unknown argument: " + self.unit + ". Choose 'word' or 'char' as a unit.")

    def fit(self, texts : list[str]) -> None:
        """text: source for training the model"""

        if self.unit == "word":
            tokenized_text = []
            for text in texts:
                tokenized_text += ["<s>" for padding_i in range(self.N - 1)] + nltk.tokenize.word_tokenize(text)
        elif self.unit == "char":
            tokenized_text = []
            for text in texts:
                tokenized_text += ["<s>" for padding_i in range(self.N - 1)] + [character for character in text]
        else:
            raise ValueError("Unknown unit: " + self.unit + ".")

        occurences_of_context = {}

        for i in range(self.N - 1, len(tokenized_text)):
            context = tuple([tokenized_text[context_i] for context_i in range(i - self.N + 1, i)])
            target = tokenized_text[i]

            # count occurences of context
            if context not in occurences_of_context:
                occurences_of_context[context] = 1
            else:
                occurences_of_context[context] += 1

            # count occurences of N-grams
            if context not in self.probabilities_dict:
                self.probabilities_dict[context] = {}

            if target not in self.probabilities_dict[context]:
                self.probabilities_dict[context][target] = 1
            else:
                self.probabilities_dict[context][target] += 1

        # get probabilities from counts
        for context in self.probabilities_dict.keys():
            for target in self.probabilities_dict[context].keys():
                self.probabilities_dict[context][target] /= occurences_of_context[context]

    def predict(self, length : int) -> list[str]:
        """Predicts text with 'length' of units."""

        context = tuple(["<s>" for context_i in range(self.N - 1)])
        generated_text = []

        while len(generated_text) < length:
            next_unit = self._next_unit(context)

            context_addition = []
            if self.N > 1 :
                context_addition.append(next_unit)
            context = tuple([context[context_i + 1] for context_i in range(self.N - 2)] + context_addition)

            if next_unit != "<s>":
                generated_text.append(next_unit)

        return generated_text

    def _next_unit(self, context : tuple[str]) -> str:
        if context not in self.probabilities_dict:
            return "<s>"

        units = []
        probabilities = []

        for unit, probability in self.probabilities_dict[context].items():
            units.append(unit)
            probabilities.append(probability)

        return random.choices(units, probabilities)[0]

#### 4 A

In [111]:
all_spoken_lines = []
for play_name in play_names:
    all_spoken_lines += play_info[play_name].spoken_lines

**word unigrams**

In [112]:
model = Ngram_model(1, "word")
model.fit(all_spoken_lines)
prediction = model.predict(20)

print(" ".join(prediction))

, bring heap says Was , for ! That I you second shall The a not stabbed , chide ,


**word bigrams**

In [125]:
model = Ngram_model(2, "word")
model.fit(all_spoken_lines)
prediction = model.predict(20)

print(" ".join(prediction))

Stands he well done , and being down , When good Tubal ; for her 'T is -- a heavy


**word trigrams**

In [114]:
model = Ngram_model(3, "word")
model.fit(all_spoken_lines)
prediction = model.predict(20)

print(" ".join(prediction))

Your servants ever And bring these gentlemen , And with such familiar instances , name ? Whither am I mad


The bigger the N, the more fluent the generated sentence sounds.

#### 4 B **Generating just from the first 50 lines of Hamlet**

In [127]:
model = Ngram_model(3, "word")
model.fit(play_info["hamlet.xml"].spoken_lines[:50])
prediction = model.predict(20)

print(" ".join(prediction))

What we have two nights seen . Holla ! Bernardo ! He may approve our eyes and speak to it


If we have very little text to learn from, there wont be many possible continuation to a given context, so the model will just repeat the sentences from the learned material. The bigger the model (the more learning material is available to the model), the more continuations for a given context. Therefore the probability of the model repeating sentences from the source, or even repeating itself are much lower.

In the above example we can see, that most of the line is a copy of a line from the beginning of Hamlet.

#### 4 C

What happens if you generate text character-by-character (letter-by-letter) instead of word-by-word?

New words that sound like english but have no meaning happen!

**character bigrams**

In [116]:
model = Ngram_model(2, "char")
model.fit(all_spoken_lines)
prediction = model.predict(100)

print("".join(prediction))

Thas, ngose he ad sod the ounthis thespeansthelerobe chy, mowhea; mbes.Whave w!Theve omuar ovipse, t


**character trigrams**

In [129]:
model = Ngram_model(3, "char")
model.fit(all_spoken_lines)
prediction = model.predict(100)

print("".join(prediction))

Thalty rethe suff. And thatere, my wasou suciall nowThe damet wore your firs tan thall your hat.How;


**character 4-grams**

In [118]:
model = Ngram_model(4, "char")
model.fit(all_spoken_lines)
prediction = model.predict(100)

print("".join(prediction))

There acturesar els,More me,Post lie to ence of thou the withat cue.I am in they to come servicessiu


## Task 5

Naive Bayes model.

In [119]:
class Naive_bayes_model:
    """Naive Bayes model.

    Needs to be trained by calling 'fit()' before usage.
    """
    def __init__(self) -> None:
        self.categories = 0
        self.tokens_in_category = {}
        self.category_probability = []
        self.category_size_in_tokens = []
        self.vocab = set()

    def fit(self, data: list[list[str]]) -> None:
        """data: Training data.

        List of lists of data of each category.

        Number of categories determined by the length of 'data'.
        """
        # init
        categories = len(data)
        self.categories = categories
        self.category_size_in_tokens = [0 for i in range(categories)]
        self.category_probability = [0 for i in range(categories)]

        for category in range(categories):
            # init dictionary for counting token frequencies
            self.tokens_in_category[category] : dict[str, int] = {}

            for line in data[category]:
                # count lines of data for each category
                self.category_probability[category] += 1

                for token in nltk.tokenize.word_tokenize(line):
                    if token not in self.vocab:
                        self.vocab.add(token)

                    if token not in self.tokens_in_category[category]:
                        self.tokens_in_category[category][token] = 0
                    self.tokens_in_category[category][token] += 1

                    self.category_size_in_tokens[category] += 1

        # number of data in all categories
        sum_of_categories = sum(self.category_probability)

        # calculate probabilities
        for category in range(categories):
            self.category_probability[category] /= sum_of_categories


    def predict(self, line : str) -> int:
        """Predict a class of the input line."""

        # init 0 probability for each category
        predictions = [0 for i in range(self.categories)]

        for category in range(self.categories):
            # init prediction with category probability
            prediction = self.category_probability[category]

            for token in nltk.tokenize.word_tokenize(line):
                # ignore tokes not included in vocabulary
                if token not in self.vocab:
                    continue

                token_count_in_category = 0
                if token in self.tokens_in_category[category]:
                    token_count_in_category = self.tokens_in_category[category][token]

                # multiply the prediction by smoothed probability of a word being in the category
                prediction *= ((token_count_in_category + 1) / (self.category_size_in_tokens[category] + len(self.vocab)))

            predictions[category] = prediction

        return np.argmax(predictions)

In [120]:
def evaluate(model : Naive_bayes_model, test_data : list[list[str]]) -> None:
    overall_correct = 0
    overall_length = 0

    for category in range(len(test_data)):
        correct = 0
        for test_line in test_data[category]:
            prediction = model.predict(test_line)

            if prediction == category:
                correct += 1

        print(category, " : ", (correct / len(test_data[category])))
        overall_correct += correct
        overall_length += len(test_data[category])

    print(overall_correct/ overall_length)
    print()

Split train and test sets by the acts.

In [121]:
train_play = []
test_play = []

for play_name in play_names:
    train_play.append(play_info[play_name].spoken_lines[:play_info[play_name].lines_per_act[3]])
    test_play.append(play_info[play_name].spoken_lines[play_info[play_name].lines_per_act[3]:])

Here, I am attempting to find the pair of the plays, that are the best distinguished by my naive bayes classifier.

In [122]:
for i in range(8):
    for j in range(i + 1, 8):
        print(play_names[i])
        print(play_names[j])
        model = Naive_bayes_model()
        model.fit([train_play[i], train_play[j]])
        evaluate(model, [test_play[i], test_play[j]])

a_and_c.xml
dream.xml
0  :  0.8980769230769231
1  :  0.4117647058823529
0.6793650793650794

a_and_c.xml
hamlet.xml
0  :  0.6096153846153847
1  :  0.6914600550964187
0.6573033707865169

a_and_c.xml
j_caesar.xml
0  :  0.7423076923076923
1  :  0.56
0.6659217877094972

a_and_c.xml
macbeth.xml
0  :  0.8288461538461539
1  :  0.345088161209068
0.6194111232279171

a_and_c.xml
merchant.xml
0  :  0.7692307692307693
1  :  0.6172839506172839
0.7109004739336493

a_and_c.xml
othello.xml
0  :  0.6365384615384615
1  :  0.7227191413237924
0.6811862835959221

a_and_c.xml
r_and_j.xml
0  :  0.7423076923076923
1  :  0.7214611872146118
0.732776617954071

dream.xml
hamlet.xml
0  :  0.3247058823529412
1  :  0.9201101928374655
0.7002606429192006

dream.xml
j_caesar.xml
0  :  0.5129411764705882
1  :  0.8506666666666667
0.67125

dream.xml
macbeth.xml
0  :  0.6094117647058823
1  :  0.6926952141057935
0.6496350364963503

dream.xml
merchant.xml
0  :  0.48
1  :  0.7592592592592593
0.6008010680907877

dream.xml
othel

The pair with the best accuracy is *Hamlet* and *Julius Caesar* with accuracy of *75.02%* of correctly classifying a line. The accuracy of recognising a *Hamlet* line correctly is *83.61%*, but it's only *58.4%* for recognising *Julius Caesar* line correctly.

So I actually think the two most easily distinguishable plays are *Julius Caesar* and *Othello*, with accuracy of *63.46% and 79.79%*. The overall accuracy of line classification is *73.23%*.


**Classifying between all 8 plays**

Here it can be nicely seen, how longer plays have greater accuracy.

In [123]:
model = Naive_bayes_model()
model.fit(train_play)
evaluate(model, test_play)

0  :  0.3596153846153846
1  :  0.1388235294117647
2  :  0.4462809917355372
3  :  0.32266666666666666
4  :  0.09319899244332494
5  :  0.28703703703703703
6  :  0.37209302325581395
7  :  0.45662100456621
0.32651434643995747

